<a href="https://colab.research.google.com/github/shashi3876/kaggle/blob/main/PodcastListenPredictLightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

THis notebook contains the work done for the Kaggel playgorund competition named Predict Podcast Listening Time. Mainly I'm using LIghtGBM in this notebook for the predcitions.

In [9]:
# imports

import pandas as pd

from sklearn.model_selection import KFold
from sklearn.preprocessing import TargetEncoder
import lightgbm as lgb
import numpy as np

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

In [4]:
train.head()

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031


# Preprocess Data

First I'm filling the null values of train and test set.

In [5]:
train.isna().sum()

,0
id,0
Podcast_Name,0
Episode_Title,1
Episode_Length_minutes,35458
Genre,1
Host_Popularity_percentage,1
Publication_Day,1
Publication_Time,1
Guest_Popularity_percentage,59152
Number_of_Ads,2


In [6]:
train['Episode_Length_minutes'].fillna(train['Episode_Length_minutes'].median(), inplace=True)
test['Episode_Length_minutes'].fillna(train['Episode_Length_minutes'].median(), inplace=True)

train['Number_of_Ads'].fillna(train['Number_of_Ads'].median(), inplace=True)
train['Guest_Popularity_percentage'].fillna(train['Guest_Popularity_percentage'].median(), inplace=True)


test['Guest_Popularity_percentage'].fillna(test['Guest_Popularity_percentage'].median(), inplace=True)

<ipython-input-6-099d9f371e57>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['Episode_Length_minutes'].fillna(train['Episode_Length_minutes'].median(), inplace=True)
<ipython-input-6-099d9f371e57>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df

Now doing the label encoding on the categorical data

In [7]:
categorical_features = [
    'Podcast_Name',
    'Genre',
    'Publication_Day',
    'Publication_Time',
    'Episode_Sentiment'
]
from sklearn.preprocessing import LabelEncoder

# Encoder for categorical data
label_encoders = {col: LabelEncoder() for col in categorical_features}

# Apply LabelEncoder to each categorical column
for col in categorical_features:
    train[col] = label_encoders[col].fit_transform(train[col])
    test[col] = label_encoders[col].transform(test[col])

    # Converting to category type
    train[col] = train[col].astype('category')
    test[col] = test[col].astype('category')

In [8]:
train['Episode_Num'] = train['Episode_Title'].str[8:].astype('category')
test['Episode_Num'] = test['Episode_Title'].str[8:].astype('category')

train = train.drop(columns=['Episode_Title','id'])
test = test.drop(columns=['Episode_Title','id'])

In [9]:
train.head()

,Podcast_Name,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes,Episode_Num
0,35,63.88,9,74.81,4,3,53.46,0.0,2,31.41998,98
1,25,119.80,1,66.95,2,0,75.95,2.0,0,88.01241,26
2,41,73.90,2,69.97,5,1,8.97,0.0,0,44.92531,16
3,11,67.17,8,57.22,1,2,78.70,2.0,2,46.27824,45
4,32,110.51,3,80.07,1,0,58.68,3.0,1,75.61031,86


In [10]:
train.columns

Index(['Podcast_Name', 'Episode_Length_minutes', 'Genre',
       'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time',
       'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment',
       'Listening_Time_minutes', 'Episode_Num'],
      dtype='object')

In [11]:
categorical_cols = ['Podcast_Name', 'Genre', 'Publication_Day', 'Publication_Time','Episode_Sentiment', 'Episode_Num']

# Build the model

In [12]:
X = train.drop(columns=['Listening_Time_minutes'])
y = train['Listening_Time_minutes']

In [13]:
X.columns

Index(['Podcast_Name', 'Episode_Length_minutes', 'Genre',
       'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time',
       'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment',
       'Episode_Num'],
      dtype='object')

In [14]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import TargetEncoder
import lightgbm as lgb
import numpy as np

categorical_cols = ['Podcast_Name', 'Genre', 'Publication_Day', 'Publication_Time','Episode_Sentiment', 'Episode_Num']
cv = KFold(5, random_state=42, shuffle=True)
y_pred = np.zeros(len(test))

for idx_train, idx_valid in cv.split(X, y):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    X_test = test[X.columns].copy()

    encoded_columns = categorical_cols
    encoder = TargetEncoder(random_state=42)

    X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
    X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
    X_test[encoded_columns] = encoder.transform(X_test[encoded_columns])


    model = lgb.LGBMRegressor(
        n_iter=2000,
        max_depth=-1,
        num_leaves=1024,
        colsample_bytree=0.7,
        learning_rate=0.03,
        objective='l2',
        metric='rmse',
        verbosity=-1,
        max_bin=1024,
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        callbacks=[
            lgb.log_evaluation(100),
            lgb.early_stopping(stopping_rounds=100)
            ],
    )

    y_pred += model.predict(X_test)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/_array_api.py:399: RuntimeWarning: invalid value encountered in cast
  return x.astype(dtype, copy=copy, casting=casting)


ValueError: Input y contains NaN.

In [ ]:
pred_lgbm = y_pred /5
sample_submission=pd.read_csv('sample_submission.csv')
submission_lgbm = pd.DataFrame({'id': sample_submission.id, 'Listening_Time_minutes' : pred_lgbm})
submission_lgbm.to_csv('submission_lgbm_2.csv', index=False)

# Fill the missing values using group wise impuation

In [25]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

# Fill train missing values by mean per Podcast_Name group
train['Episode_Length_minutes'] = train['Episode_Length_minutes'].fillna(
    train.groupby('Podcast_Name')['Episode_Length_minutes'].transform('mean')
)

train['Number_of_Ads'] = train['Number_of_Ads'].fillna(
    train.groupby('Podcast_Name')['Number_of_Ads'].transform('mean')
)

train['Guest_Popularity_percentage'] = train['Guest_Popularity_percentage'].fillna(
    train.groupby('Podcast_Name')['Guest_Popularity_percentage'].transform('mean')
)

# For test set: use group means from the train set (safe approach)
podcast_means = train.groupby('Podcast_Name')[['Episode_Length_minutes', 'Guest_Popularity_percentage']].mean()

# Map those means to the test set
test['Episode_Length_minutes'] = test['Episode_Length_minutes'].fillna(
    test['Podcast_Name'].map(podcast_means['Episode_Length_minutes'])
)

test['Guest_Popularity_percentage'] = test['Guest_Popularity_percentage'].fillna(
    test['Podcast_Name'].map(podcast_means['Guest_Popularity_percentage'])

)

In [26]:
train['Episode_Num'] = train['Episode_Title'].str[8:].astype('category')
test['Episode_Num'] = test['Episode_Title'].str[8:].astype('category')

train = train.drop(columns=['Episode_Title','id'])
test = test.drop(columns=['Episode_Title','id'])

X = train.drop(columns=['Listening_Time_minutes'])
y = train['Listening_Time_minutes']

categorical_cols = ['Podcast_Name', 'Genre', 'Publication_Day', 'Publication_Time','Episode_Sentiment', 'Episode_Num']
cv = KFold(5, random_state=42, shuffle=True)
y_pred = np.zeros(len(test))

for idx_train, idx_valid in cv.split(X, y):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    X_test = test[X.columns].copy()

    encoded_columns = categorical_cols
    encoder = TargetEncoder(random_state=42)

    X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
    X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
    X_test[encoded_columns] = encoder.transform(X_test[encoded_columns])


    model = lgb.LGBMRegressor(
        n_iter=2000,
        max_depth=-1,
        num_leaves=1024,
        colsample_bytree=0.7,
        learning_rate=0.03,
        objective='l2',
        metric='rmse',
        verbosity=-1,
        max_bin=1024,
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        callbacks=[
            lgb.log_evaluation(100),
            lgb.early_stopping(stopping_rounds=100)
            ],
    )

    y_pred += model.predict(X_test)


<ipython-input-26-bb1d328f2c99>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-26-bb1d328f2c99>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.2819
[200]	valid_0's rmse: 12.9324
[300]	valid_0's rmse: 12.9006
[400]	valid_0's rmse: 12.8828
[500]	valid_0's rmse: 12.8695
[600]	valid_0's rmse: 12.8618
[700]	valid_0's rmse: 12.8545
[800]	valid_0's rmse: 12.8491
[900]	valid_0's rmse: 12.8426
[1000]	valid_0's rmse: 12.8371
[1100]	valid_0's rmse: 12.8333
[1200]	valid_0's rmse: 12.8286
[1300]	valid_0's rmse: 12.8241
[1400]	valid_0's rmse: 12.8201
[1500]	valid_0's rmse: 12.8186
[1600]	valid_0's rmse: 12.8165
[1700]	valid_0's rmse: 12.8142
[1800]	valid_0's rmse: 12.8124
[1900]	valid_0's rmse: 12.8103
[2000]	valid_0's rmse: 12.8095
Did not meet early stopping. Best iteration is:
[1998]	valid_0's rmse: 12.8093


<ipython-input-26-bb1d328f2c99>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-26-bb1d328f2c99>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.33
[200]	valid_0's rmse: 12.9796
[300]	valid_0's rmse: 12.9469
[400]	valid_0's rmse: 12.937
[500]	valid_0's rmse: 12.9258
[600]	valid_0's rmse: 12.9155
[700]	valid_0's rmse: 12.9068
[800]	valid_0's rmse: 12.9008
[900]	valid_0's rmse: 12.8927
[1000]	valid_0's rmse: 12.8891
[1100]	valid_0's rmse: 12.8838
[1200]	valid_0's rmse: 12.8803
[1300]	valid_0's rmse: 12.8775
[1400]	valid_0's rmse: 12.8745
[1500]	valid_0's rmse: 12.8734
[1600]	valid_0's rmse: 12.8729
[1700]	valid_0's rmse: 12.8702
[1800]	valid_0's rmse: 12.869
[1900]	valid_0's rmse: 12.8678
[2000]	valid_0's rmse: 12.8669
Did not meet early stopping. Best iteration is:
[1980]	valid_0's rmse: 12.8666


<ipython-input-26-bb1d328f2c99>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-26-bb1d328f2c99>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.3282
[200]	valid_0's rmse: 12.9766
[300]	valid_0's rmse: 12.9412
[400]	valid_0's rmse: 12.9255
[500]	valid_0's rmse: 12.9114
[600]	valid_0's rmse: 12.9016
[700]	valid_0's rmse: 12.892
[800]	valid_0's rmse: 12.8854
[900]	valid_0's rmse: 12.8802
[1000]	valid_0's rmse: 12.8741
[1100]	valid_0's rmse: 12.8706
[1200]	valid_0's rmse: 12.8655
[1300]	valid_0's rmse: 12.8607
[1400]	valid_0's rmse: 12.8573
[1500]	valid_0's rmse: 12.8548
[1600]	valid_0's rmse: 12.8519
[1700]	valid_0's rmse: 12.8486
[1800]	valid_0's rmse: 12.8472
[1900]	valid_0's rmse: 12.8441
[2000]	valid_0's rmse: 12.8433
Did not meet early stopping. Best iteration is:
[1999]	valid_0's rmse: 12.8433


<ipython-input-26-bb1d328f2c99>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-26-bb1d328f2c99>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.3362
[200]	valid_0's rmse: 12.9746
[300]	valid_0's rmse: 12.939
[400]	valid_0's rmse: 12.923
[500]	valid_0's rmse: 12.9113
[600]	valid_0's rmse: 12.9029
[700]	valid_0's rmse: 12.8939
[800]	valid_0's rmse: 12.8856
[900]	valid_0's rmse: 12.8751
[1000]	valid_0's rmse: 12.8697
[1100]	valid_0's rmse: 12.8665
[1200]	valid_0's rmse: 12.8617
[1300]	valid_0's rmse: 12.8569
[1400]	valid_0's rmse: 12.8514
[1500]	valid_0's rmse: 12.85
[1600]	valid_0's rmse: 12.8484
[1700]	valid_0's rmse: 12.8448
[1800]	valid_0's rmse: 12.8441
[1900]	valid_0's rmse: 12.8429
Early stopping, best iteration is:
[1896]	valid_0's rmse: 12.8426


<ipython-input-26-bb1d328f2c99>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-26-bb1d328f2c99>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.276
[200]	valid_0's rmse: 12.9335
[300]	valid_0's rmse: 12.898
[400]	valid_0's rmse: 12.8803
[500]	valid_0's rmse: 12.8693
[600]	valid_0's rmse: 12.8595
[700]	valid_0's rmse: 12.8523
[800]	valid_0's rmse: 12.8434
[900]	valid_0's rmse: 12.8371
[1000]	valid_0's rmse: 12.832
[1100]	valid_0's rmse: 12.8283
[1200]	valid_0's rmse: 12.8248
[1300]	valid_0's rmse: 12.8203
[1400]	valid_0's rmse: 12.8169
[1500]	valid_0's rmse: 12.8152
[1600]	valid_0's rmse: 12.8137
[1700]	valid_0's rmse: 12.8104
[1800]	valid_0's rmse: 12.8067
[1900]	valid_0's rmse: 12.8048
[2000]	valid_0's rmse: 12.8041
Did not meet early stopping. Best iteration is:
[1993]	valid_0's rmse: 12.8039


In [37]:
pred_lgbm = y_pred /5
sample_submission=pd.read_csv('sample_submission.csv')
submission_lgbm = pd.DataFrame({'id': sample_submission.id, 'Listening_Time_minutes' : pred_lgbm})
submission_lgbm.to_csv('submission_lgbm_3.csv', index=False)

In [44]:
test_all = pd.read_csv('test.csv')
sample = submission_lgbm.merge(test_all, on='id')
print(len(sample))
mask = sample['Episode_Length_minutes'].notna()

sample.loc[mask, 'Listening_Time_minutes'] = np.minimum(
    sample.loc[mask, 'Listening_Time_minutes'],
    sample.loc[mask, 'Episode_Length_minutes']
)

sample[['id', 'Listening_Time_minutes']].to_csv('submission_lgbm_4.csv', index=False)

250000


In [42]:
len(test_all)
sample = submission_lgbm.merge(test_all, on='id').head()
print(len(sample))

5


In [35]:
len(sample)

5

# Add combined categorical features

In [17]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

train['Episode_Num'] = train['Episode_Title'].str[8:].astype('category')
test['Episode_Num'] = test['Episode_Title'].str[8:].astype('category')

train = train.drop(columns=['Episode_Title','id'])
test = test.drop(columns=['Episode_Title','id'])

# Fill train missing values by mean per Podcast_Name group
train['Episode_Length_minutes'] = train['Episode_Length_minutes'].fillna(
    train.groupby('Podcast_Name')['Episode_Length_minutes'].transform('mean')
)

train['Number_of_Ads'] = train['Number_of_Ads'].fillna(
    train.groupby('Podcast_Name')['Number_of_Ads'].transform('mean')
)

train['Guest_Popularity_percentage'] = train['Guest_Popularity_percentage'].fillna(
    train.groupby('Podcast_Name')['Guest_Popularity_percentage'].transform('mean')
)

# For test set: use group means from the train set (safe approach)
podcast_means = train.groupby('Podcast_Name')[['Episode_Length_minutes', 'Guest_Popularity_percentage']].mean()

# Map those means to the test set
test['Episode_Length_minutes'] = test['Episode_Length_minutes'].fillna(
    test['Podcast_Name'].map(podcast_means['Episode_Length_minutes'])
)

test['Guest_Popularity_percentage'] = test['Guest_Popularity_percentage'].fillna(
    test['Podcast_Name'].map(podcast_means['Guest_Popularity_percentage'])

)



In [20]:
test.describe()

,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads
count,2.500000e+05,250000.000000,250000.000000,250000.000000
mean,3.785137e+02,59.716491,52.197435,1.355852
std,1.569724e+05,22.880028,25.518748,4.274399
min,2.470000e+00,2.490000,0.000000,0.000000
25%,3.937000e+01,39.250000,34.550000,0.000000
50%,6.446000e+01,59.900000,52.430000,1.000000
75%,9.051000e+01,79.390000,70.990000,2.000000
max,7.848626e+07,117.760000,116.820000,2063.000000


In [21]:
from tqdm import tqdm
cat_cols = [
    'Podcast_Name', 'Genre', 'Publication_Day',
    'Publication_Time', 'Episode_Sentiment', 'Episode_Num'
]


combined_features =  cat_cols

# Create combinations of 2 features (can go 2, 3, 4 but heavy)
from itertools import combinations
pair_size = [2,3]

for r in pair_size:
    combos = list(combinations(combined_features, r))
    for cols in tqdm(combos):
        col_name = '_'.join(cols)
        train[col_name] = train[list(cols)].astype(str).agg('_'.join, axis=1).astype('category')
        test[col_name] = test[list(cols)].astype(str).agg('_'.join, axis=1).astype('category')

100%|██████████| 20/20 [01:42<00:00,  5.15s/it]


In [22]:
train.columns

Index(['Podcast_Name', 'Episode_Length_minutes', 'Genre',
       'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time',
       'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment',
       'Listening_Time_minutes', 'Episode_Num', 'Podcast_Name_Genre',
       'Podcast_Name_Publication_Day', 'Podcast_Name_Publication_Time',
       'Podcast_Name_Episode_Sentiment', 'Podcast_Name_Episode_Num',
       'Genre_Publication_Day', 'Genre_Publication_Time',
       'Genre_Episode_Sentiment', 'Genre_Episode_Num',
       'Publication_Day_Publication_Time', 'Publication_Day_Episode_Sentiment',
       'Publication_Day_Episode_Num', 'Publication_Time_Episode_Sentiment',
       'Publication_Time_Episode_Num', 'Episode_Sentiment_Episode_Num',
       'Podcast_Name_Genre_Publication_Day',
       'Podcast_Name_Genre_Publication_Time',
       'Podcast_Name_Genre_Episode_Sentiment',
       'Podcast_Name_Genre_Episode_Num',
       'Podcast_Name_Publication_Day_Publication_Time',
   

In [23]:


X = train.drop(columns=['Listening_Time_minutes'])
y = train['Listening_Time_minutes']

# Define numeric columns
num_cols = ['Listening_Time_minutes','Episode_Length_minutes', 'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Number_of_Ads']

# Get categorical columns as the rest of the columns in train
categorical_cols = [col for col in train.columns if col not in num_cols]

'''
categorical_cols = ['Podcast_Name',  'Genre', 'Publication_Day', 'Publication_Time',
        'Episode_Sentiment',
        'Episode_Num', 'Podcast_Name_Genre',
       'Podcast_Name_Publication_Day', 'Podcast_Name_Publication_Time',
       'Podcast_Name_Episode_Sentiment', 'Podcast_Name_Episode_Num',
       'Genre_Publication_Day', 'Genre_Publication_Time',
       'Genre_Episode_Sentiment', 'Genre_Episode_Num',
       'Publication_Day_Publication_Time', 'Publication_Day_Episode_Sentiment',
       'Publication_Day_Episode_Num', 'Publication_Time_Episode_Sentiment',
       'Publication_Time_Episode_Num', 'Episode_Sentiment_Episode_Num']
'''
cv = KFold(5, random_state=42, shuffle=True)
y_pred = np.zeros(len(test))

for idx_train, idx_valid in cv.split(X, y):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    X_test = test[X.columns].copy()

    encoded_columns = categorical_cols
    encoder = TargetEncoder(random_state=42)

    X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
    X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
    X_test[encoded_columns] = encoder.transform(X_test[encoded_columns])


    model = lgb.LGBMRegressor(
        n_iter=2000,
        max_depth=-1,
        num_leaves=1024,
        colsample_bytree=0.7,
        learning_rate=0.03,
        objective='l2',
        metric='rmse',
        verbosity=-1,
        max_bin=1024,
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        callbacks=[
            lgb.log_evaluation(100),
            lgb.early_stopping(stopping_rounds=100)
            ],
    )

    y_pred += model.predict(X_test)


<ipython-input-23-0a39f8fa054f>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-23-0a39f8fa054f>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.2584
[200]	valid_0's rmse: 12.9709
[300]	valid_0's rmse: 12.949
[400]	valid_0's rmse: 12.9398
[500]	valid_0's rmse: 12.9342
[600]	valid_0's rmse: 12.9321
[700]	valid_0's rmse: 12.9321
[800]	valid_0's rmse: 12.9284
[900]	valid_0's rmse: 12.9258
[1000]	valid_0's rmse: 12.9244
[1100]	valid_0's rmse: 12.9228
[1200]	valid_0's rmse: 12.9229
Early stopping, best iteration is:
[1130]	valid_0's rmse: 12.9216


<ipython-input-23-0a39f8fa054f>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-23-0a39f8fa054f>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.3115
[200]	valid_0's rmse: 13.0266
[300]	valid_0's rmse: 13.0041
[400]	valid_0's rmse: 12.9942
[500]	valid_0's rmse: 12.9908
[600]	valid_0's rmse: 12.9893
[700]	valid_0's rmse: 12.9878
Early stopping, best iteration is:
[643]	valid_0's rmse: 12.9872


<ipython-input-23-0a39f8fa054f>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-23-0a39f8fa054f>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.3081
[200]	valid_0's rmse: 13.0234
[300]	valid_0's rmse: 12.9973
[400]	valid_0's rmse: 12.9865
[500]	valid_0's rmse: 12.9799
[600]	valid_0's rmse: 12.9788
[700]	valid_0's rmse: 12.9759
[800]	valid_0's rmse: 12.9738
[900]	valid_0's rmse: 12.9723
[1000]	valid_0's rmse: 12.9702
[1100]	valid_0's rmse: 12.9699
[1200]	valid_0's rmse: 12.968
[1300]	valid_0's rmse: 12.9674
[1400]	valid_0's rmse: 12.968
Early stopping, best iteration is:
[1376]	valid_0's rmse: 12.9673


<ipython-input-23-0a39f8fa054f>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-23-0a39f8fa054f>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.3095
[200]	valid_0's rmse: 13.0171
[300]	valid_0's rmse: 12.9873
[400]	valid_0's rmse: 12.9813
[500]	valid_0's rmse: 12.9779
[600]	valid_0's rmse: 12.9758
[700]	valid_0's rmse: 12.9732
[800]	valid_0's rmse: 12.9727
[900]	valid_0's rmse: 12.9708
[1000]	valid_0's rmse: 12.9694
[1100]	valid_0's rmse: 12.9673
[1200]	valid_0's rmse: 12.9666
[1300]	valid_0's rmse: 12.9663
[1400]	valid_0's rmse: 12.9643
Early stopping, best iteration is:
[1388]	valid_0's rmse: 12.9641


<ipython-input-23-0a39f8fa054f>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-23-0a39f8fa054f>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.2648
[200]	valid_0's rmse: 12.9898
[300]	valid_0's rmse: 12.9629
[400]	valid_0's rmse: 12.9545
[500]	valid_0's rmse: 12.9513
[600]	valid_0's rmse: 12.948
[700]	valid_0's rmse: 12.9469
[800]	valid_0's rmse: 12.9449
[900]	valid_0's rmse: 12.9453
Early stopping, best iteration is:
[808]	valid_0's rmse: 12.9444


In [24]:
pred_lgbm = y_pred /5
sample_submission=pd.read_csv('sample_submission.csv')
submission_lgbm = pd.DataFrame({'id': sample_submission.id, 'Listening_Time_minutes' : pred_lgbm})
submission_lgbm.to_csv('submission_lgbm_5.csv', index=False)

In [ ]:
train.columns

# LightGBM with clipping in input features

In [19]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

# Fill train missing values by mean per Podcast_Name group
train['Episode_Length_minutes'] = train['Episode_Length_minutes'].fillna(
    train.groupby('Podcast_Name')['Episode_Length_minutes'].transform('mean')
)

train['Number_of_Ads'] = train['Number_of_Ads'].fillna(
    train.groupby('Podcast_Name')['Number_of_Ads'].transform('mean')
)

train['Guest_Popularity_percentage'] = train['Guest_Popularity_percentage'].fillna(
    train.groupby('Podcast_Name')['Guest_Popularity_percentage'].transform('mean')
)

# For test set: use group means from the train set (safe approach)
podcast_means = train.groupby('Podcast_Name')[['Episode_Length_minutes', 'Guest_Popularity_percentage']].mean()

# Map those means to the test set
test['Episode_Length_minutes'] = test['Episode_Length_minutes'].fillna(
    test['Podcast_Name'].map(podcast_means['Episode_Length_minutes'])
)

test['Guest_Popularity_percentage'] = test['Guest_Popularity_percentage'].fillna(
    test['Podcast_Name'].map(podcast_means['Guest_Popularity_percentage'])

)

In [24]:
train.describe()

,id,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Listening_Time_minutes
count,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000
mean,374999.500000,64.502111,59.859901,52.233466,1.347916,45.437406
std,216506.495284,31.000037,22.873098,25.534173,1.110998,27.138306
min,0.000000,0.000000,1.300000,0.000000,0.000000,0.000000
25%,187499.750000,39.420000,39.410000,34.550000,0.000000,23.178350
50%,374999.500000,64.423470,60.050000,52.431959,1.000000,43.379460
75%,562499.250000,90.310000,79.530000,71.040000,2.000000,64.811580
max,749999.000000,120.000000,119.460000,119.910000,4.000000,119.970000


In [23]:
test.describe()

,id,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000
mean,874999.500000,64.539313,59.716491,52.197435,1.347276
std,72168.927986,31.006801,22.880028,25.518748,1.112691
min,750000.000000,2.470000,2.490000,0.000000,0.000000
25%,812499.750000,39.370000,39.250000,34.550000,0.000000
50%,874999.500000,64.460000,59.900000,52.430000,1.000000
75%,937499.250000,90.510000,79.390000,70.990000,2.000000
max,999999.000000,120.000000,117.760000,116.820000,4.000000


In [22]:
train['Number_of_Ads'] = train['Number_of_Ads'].clip(upper=4)
train['Episode_Length_minutes'] = train['Episode_Length_minutes'].clip(upper=120)

test['Number_of_Ads'] = test['Number_of_Ads'].clip(upper=4)
test['Episode_Length_minutes'] = test['Episode_Length_minutes'].clip(upper=120)

In [15]:
train['Episode_Num'] = train['Episode_Title'].str[8:].astype('category')
test['Episode_Num'] = test['Episode_Title'].str[8:].astype('category')

train = train.drop(columns=['Episode_Title','id'])
test = test.drop(columns=['Episode_Title','id'])

X = train.drop(columns=['Listening_Time_minutes'])
y = train['Listening_Time_minutes']

categorical_cols = ['Podcast_Name', 'Genre', 'Publication_Day', 'Publication_Time','Episode_Sentiment', 'Episode_Num']
cv = KFold(5, random_state=42, shuffle=True)
y_pred = np.zeros(len(test))

for idx_train, idx_valid in cv.split(X, y):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    X_test = test[X.columns].copy()

    encoded_columns = categorical_cols
    encoder = TargetEncoder(random_state=42)

    X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
    X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
    X_test[encoded_columns] = encoder.transform(X_test[encoded_columns])


    model = lgb.LGBMRegressor(
        n_iter=2000,
        max_depth=-1,
        num_leaves=1024,
        colsample_bytree=0.7,
        learning_rate=0.03,
        objective='l2',
        metric='rmse',
        verbosity=-1,
        max_bin=1024,
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        callbacks=[
            lgb.log_evaluation(100),
            lgb.early_stopping(stopping_rounds=100)
            ],
    )

    y_pred += model.predict(X_test)


<ipython-input-15-bb1d328f2c99>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-15-bb1d328f2c99>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.2819
[200]	valid_0's rmse: 12.9324
[300]	valid_0's rmse: 12.9006
[400]	valid_0's rmse: 12.8828
[500]	valid_0's rmse: 12.8695
[600]	valid_0's rmse: 12.8618
[700]	valid_0's rmse: 12.8545
[800]	valid_0's rmse: 12.8491
[900]	valid_0's rmse: 12.8426
[1000]	valid_0's rmse: 12.8371
[1100]	valid_0's rmse: 12.8333
[1200]	valid_0's rmse: 12.8286
[1300]	valid_0's rmse: 12.8241
[1400]	valid_0's rmse: 12.8201
[1500]	valid_0's rmse: 12.8186
[1600]	valid_0's rmse: 12.8165
[1700]	valid_0's rmse: 12.8142
[1800]	valid_0's rmse: 12.8124
[1900]	valid_0's rmse: 12.8103
[2000]	valid_0's rmse: 12.8095
Did not meet early stopping. Best iteration is:
[1998]	valid_0's rmse: 12.8093


<ipython-input-15-bb1d328f2c99>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-15-bb1d328f2c99>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.33
[200]	valid_0's rmse: 12.9796
[300]	valid_0's rmse: 12.9469
[400]	valid_0's rmse: 12.937
[500]	valid_0's rmse: 12.9258
[600]	valid_0's rmse: 12.9155
[700]	valid_0's rmse: 12.9068
[800]	valid_0's rmse: 12.9008
[900]	valid_0's rmse: 12.8927
[1000]	valid_0's rmse: 12.8891
[1100]	valid_0's rmse: 12.8838
[1200]	valid_0's rmse: 12.8803
[1300]	valid_0's rmse: 12.8775
[1400]	valid_0's rmse: 12.8745
[1500]	valid_0's rmse: 12.8734
[1600]	valid_0's rmse: 12.8729
[1700]	valid_0's rmse: 12.8702
[1800]	valid_0's rmse: 12.869
[1900]	valid_0's rmse: 12.8678
[2000]	valid_0's rmse: 12.8669
Did not meet early stopping. Best iteration is:
[1980]	valid_0's rmse: 12.8666


<ipython-input-15-bb1d328f2c99>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-15-bb1d328f2c99>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.3282
[200]	valid_0's rmse: 12.9766
[300]	valid_0's rmse: 12.9412
[400]	valid_0's rmse: 12.9255
[500]	valid_0's rmse: 12.9114
[600]	valid_0's rmse: 12.9016
[700]	valid_0's rmse: 12.892
[800]	valid_0's rmse: 12.8854
[900]	valid_0's rmse: 12.8802
[1000]	valid_0's rmse: 12.8741
[1100]	valid_0's rmse: 12.8706
[1200]	valid_0's rmse: 12.8655
[1300]	valid_0's rmse: 12.8607
[1400]	valid_0's rmse: 12.8573
[1500]	valid_0's rmse: 12.8548
[1600]	valid_0's rmse: 12.8519
[1700]	valid_0's rmse: 12.8486
[1800]	valid_0's rmse: 12.8472
[1900]	valid_0's rmse: 12.8441
[2000]	valid_0's rmse: 12.8433
Did not meet early stopping. Best iteration is:
[1999]	valid_0's rmse: 12.8433


<ipython-input-15-bb1d328f2c99>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-15-bb1d328f2c99>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.3362
[200]	valid_0's rmse: 12.9746
[300]	valid_0's rmse: 12.939
[400]	valid_0's rmse: 12.923
[500]	valid_0's rmse: 12.9113
[600]	valid_0's rmse: 12.9029
[700]	valid_0's rmse: 12.8939
[800]	valid_0's rmse: 12.8856
[900]	valid_0's rmse: 12.8751
[1000]	valid_0's rmse: 12.8697
[1100]	valid_0's rmse: 12.8665
[1200]	valid_0's rmse: 12.8617
[1300]	valid_0's rmse: 12.8569
[1400]	valid_0's rmse: 12.8514
[1500]	valid_0's rmse: 12.85
[1600]	valid_0's rmse: 12.8484
[1700]	valid_0's rmse: 12.8448
[1800]	valid_0's rmse: 12.8441
[1900]	valid_0's rmse: 12.8429
Early stopping, best iteration is:
[1896]	valid_0's rmse: 12.8426


<ipython-input-15-bb1d328f2c99>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-15-bb1d328f2c99>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.276
[200]	valid_0's rmse: 12.9335
[300]	valid_0's rmse: 12.898
[400]	valid_0's rmse: 12.8803
[500]	valid_0's rmse: 12.8693
[600]	valid_0's rmse: 12.8595
[700]	valid_0's rmse: 12.8523
[800]	valid_0's rmse: 12.8434
[900]	valid_0's rmse: 12.8371
[1000]	valid_0's rmse: 12.832
[1100]	valid_0's rmse: 12.8283
[1200]	valid_0's rmse: 12.8248
[1300]	valid_0's rmse: 12.8203
[1400]	valid_0's rmse: 12.8169
[1500]	valid_0's rmse: 12.8152
[1600]	valid_0's rmse: 12.8137
[1700]	valid_0's rmse: 12.8104
[1800]	valid_0's rmse: 12.8067
[1900]	valid_0's rmse: 12.8048
[2000]	valid_0's rmse: 12.8041
Did not meet early stopping. Best iteration is:
[1993]	valid_0's rmse: 12.8039


In [16]:
pred_lgbm = y_pred /5
sample_submission=pd.read_csv('sample_submission.csv')
submission_lgbm = pd.DataFrame({'id': sample_submission.id, 'Listening_Time_minutes' : pred_lgbm})
submission_lgbm.to_csv('submission_lgbm_clipped.csv', index=False)